# Abgabe 3 - DQN zum autonomen Fahren

Im Rahmen der dritten Abgabe im Kurs Künstliche Intelligenz erfolgte die Entwicklung und Implementierung eines Deep Q-Network (DQN) Agenten. 
Dieser soll optimale Entscheidungen in einer Autorennumgebung aus der gymnasium-Bibliothek, früher bekannt als gym, treffen.

## 1 Import & Umgebung

In [1]:
import numpy as np
import gymnasium as gym
import cv2

from agent.dqn import DQN

Als bereits fertige Bibliotheken werden einerseits *numpy* zur effizienten Berechnung von Matrizen und Vektoren, *cv2 (OpenCV)* zur Bildverarbeitung und *gymnasium* als Weiterentwicklung der gym-Bibliothek zur Simulation einer Autorennumgebung verwendet. Die Implementierung des DQN ist in eine separate Pythondatei ausgelagert. 

![Beispielbild](gym_environment.jpg)

Die Reinforcement Learning Umgebung ist Teil der Box2D-Umgebungen der gymnasium-Bibliothek und simuliert ein Rennspiel aus der Vogelperspektive. Die generierte Rennstrecke ändert sich in jeder Episode zufällig. 
- Der Aktionsraum in der Umgebung kann kontinuierlich oder diskret sein. Im kontinuierlichen Fall gibt es drei Aktionen: Lenken (von -1 für volle Linkskurve bis +1 für volle Rechtskurve), Gas geben und Bremsen. In der Implementierung wird dieser in einen diskreten Aktionsraum mit 12 möglichen Aktionen umgewandelt. 
- Als Observationen werden RGB-Bilder (96x96 Pixel, 3 Kanäle) aus einer top-down Perspektive von dem Auto und der Rennstrecke generiert.
- Die Belohnung beträgt -0,1 in jedem Frame und +1000/N für jede befahrene Tile der Rennstrecke, wobei N die Gesamtzahl der befahrenen Tiles auf der Rennstrecke ist. Zum Beispiel beträgt die Belohnung für einen Abschluss in 732 Frames 1000 - 0,1 * 732 = 926,8 Punkte.

## Training

Das Training erfolgt auf einer GPU, um den Prozess zu beschleunigen. Das Training nahm insgesamt circa 7 Stunden in Anspruch und benötigte zwischenzeitlich 11 GB Arbeitsspeicher. Die Logik des Trainings sieht folgendermaßen aus:

1. Zu Beginn wird eine Funktion process_state_image definiert, die das Eingangsbild verarbeitet. Es wird von Farb- zu Graustufen umgewandelt, um die Kanäle und damit die Anzahl der Eingabeparameter zu reduzieren.
2. Die Variable action_space definiert eine Liste von Aktionen, die der Agent in der Umgebung ausführen kann. Jede Aktion besteht aus Werten für Lenken, Gasgeben und Bremsen.
3. Die Variable state_shape gibt die Größe der Observation an, die ein 96x96 großes Graustufenbild ist.
4. Die Umgebung "CarRacing-v2" wird mit gym.make initialisiert und anschließend der DQN-Agent erstellt.
5. Der Agent wird in einer Schleife über n=100 bzw. später n=500 Episoden trainiert. In jeder Episode wird der Anfangszustand der Umgebung abgerufen. Der Agent interagiert dann mit der Umgebung, wählt Aktionen aus und sammelt Erfahrungen.
6. Während der Interaktion mit der Umgebung werden Belohnungen gesammelt und negative Belohnungen werden gezählt, um sicherzustellen, dass der Agent das Auto nicht wahllos über Grass Tiles steuert.
7. Der Agent speichert seine Erfahrungen in dem Replay-Buffer und führt alle 5 Frames ein Replay-Training durch, um das Modell zu verbessern.

In [2]:
def process_state_image(state):
    state = cv2.cvtColor(state, cv2.COLOR_BGR2GRAY)
    state = state.astype(float)
    return state / 255.0

action_space = [
    # (wheel [-1~1], gas [0~1], break [0~1])
    (-1, 1, 0.5), (0, 1, 0.5), (1, 1, 0.5),
    (-1, 1,   0), (0, 1,   0), (1, 1,   0),
    (-1, 0, 0.5), (0, 0, 0.5), (1, 0, 0.5), 
    (-1, 0,   0), (0, 0,   0), (1, 0,   0)
]
# 96x96 grayscaled image
state_shape = (96, 96, 1)

reward_history = []

env = gym.make("CarRacing-v2")
agent = DQN(action_space, state_shape)

num_episodes = 100
for episode in range(num_episodes):
    state, info = env.reset()
    # Reduce rgb image to gray scale
    state = process_state_image(state)
    total_reward = 0
    negative_reward = 0
    frame_counter = 0
    done = False

    while not done:
        action = agent.act(state)

        next_state, reward, done, truncated, info = env.step(action)

        # Reduce rgb image to gray scale
        next_state = process_state_image(next_state)

        total_reward += reward
        frame_counter += 1

        if frame_counter > 100 and reward < 0:
            # Don't get lost on grass
            negative_reward += 1
        else:
            # Reset if agent finds track
            negative_reward = 0

        if negative_reward > 10 or total_reward < 0:
            done = True # End early

        agent.remember(state, action, reward, next_state, done)
        if frame_counter % 5 == 0:
            agent.replay()

        state = next_state
    
    if episode > 0 and episode % 5 == 0:
        # Save model every 100th episode
        agent.model.save_weights(f"models/trial_{episode}/dqn")
    
    print(f"Episode: {episode+1}, Total Reward: {total_reward}")
    reward_history += [total_reward]

env.close()

Episode: 1, Total Reward: 3.714814814814845
Episode: 2, Total Reward: 1.7617363344051595
Episode: 3, Total Reward: 1.844983818770244
Episode: 4, Total Reward: 10.282733812949676
Episode: 5, Total Reward: -0.09072164948451733
Episode: 6, Total Reward: -0.09999999999998122
Episode: 7, Total Reward: 5.964846416382285
Episode: 8, Total Reward: -0.09090909090906973
Episode: 9, Total Reward: -0.03157894736840314
Episode: 10, Total Reward: -0.02727272727271865
Episode: 11, Total Reward: 10.958823529411804
Episode: 12, Total Reward: 32.22129963898913
Episode: 13, Total Reward: -0.06603773584904293
Episode: 14, Total Reward: 0.05241635687733853
Episode: 15, Total Reward: 4.9771704180064695
Episode: 16, Total Reward: -0.047311827956968616
Episode: 17, Total Reward: 5.964846416382295
Episode: 18, Total Reward: 7.487360594795579
Episode: 19, Total Reward: 5.081229773462823
Episode: 20, Total Reward: 14.922304832713701
Episode: 21, Total Reward: 6.567844522968237
Episode: 22, Total Reward: -0.05109

In [4]:
print(agent.epsilon, agent.epsilon_min)

0.00998645168764533 0.01


Nach dem ersten Training über 100 Episoden kann der durchschnittliche Reward von 2 auf circa 5 Punkte leicht verbessert werden. Das Training dazu hat über GPU 90 Minuten gedauert. Zudem sinkt der Epsilonwert durch die kontinuierliche Verringerung unter das definierte Minimum von 0.01, sodass die Exploration voll ausgeschöpft wurde und nur noch 1% der Aktionen zufällig gewählt werden. Um weitere Forschritte zu erzielen, wird das Training fortgeführt und nun 400 weitere Episoden trainiert.

In [5]:
num_episodes = 500
for episode in range(100, num_episodes):
    state, info = env.reset()
    state = process_state_image(state)
    total_reward = 0
    negative_reward = 0
    frame_counter = 0
    done = False

    while not done:
        action = agent.act(state)

        next_state, reward, done, truncated, info = env.step(action)

        next_state = process_state_image(next_state)

        total_reward += reward
        frame_counter += 1

        if frame_counter > 100 and reward < 0:
            negative_reward += 1
        else:
            negative_reward = 0

        if negative_reward > 10 or total_reward < 0:
            done = True # End early

        agent.remember(state, action, reward, next_state, done)
        if frame_counter % 5 == 0:
            agent.replay()

        state = next_state
    
    if episode > 0 and episode % 20 == 0:
        # Save every 100th episode
        agent.model.save_weights(f"models/trial_{episode}/dqn")
    
    print(f"Episode: {episode+1}, Total Reward: {total_reward}")
    reward_history += [total_reward]

env.close()

Episode: 101, Total Reward: 5.081229773462823
Episode: 102, Total Reward: 8.90000000000004
Episode: 103, Total Reward: 6.261111111111152
Episode: 104, Total Reward: 5.511295681063161
Episode: 105, Total Reward: 6.885611510791405
Episode: 106, Total Reward: 4.925641025641066
Episode: 107, Total Reward: 5.906802721088473
Episode: 108, Total Reward: 7.767924528301927
Episode: 109, Total Reward: 4.773015873015911
Episode: 110, Total Reward: 4.3798761609907455
Episode: 111, Total Reward: 7.3501845018450584
Episode: 112, Total Reward: 7.148175182481792
Episode: 113, Total Reward: 6.757142857142897
Episode: 114, Total Reward: 5.622408026755892
Episode: 115, Total Reward: 6.082130584192479
Episode: 116, Total Reward: 7.015942028985547
Episode: 117, Total Reward: 5.791891891891932
Episode: 118, Total Reward: 5.186644951140105
Episode: 119, Total Reward: 5.491891891891926
Episode: 120, Total Reward: 4.403875968992287
Episode: 121, Total Reward: -0.006329113924033097
Episode: 122, Total Reward: 7

Nach 215 Episoden kam es leider zu einem Out-Of-Memory-Fehler, sodass das Training ab Episode 200 wieder aufgenommen wurde. Der Replaybuffer kann jedoch nicht wiederhergestellt werden.

In [3]:
def process_state_image(state):
    state = cv2.cvtColor(state, cv2.COLOR_BGR2GRAY)
    state = state.astype(float)
    return state / 255.0

action_space = [
    (-1, 1, 0.5), (0, 1, 0.5), (1, 1, 0.5), # Action Space Structure
    (-1, 1,   0), (0, 1,   0), (1, 1,   0), # (Steering Wheel, Gas, Break)
    (-1, 0, 0.5), (0, 0, 0.5), (1, 0, 0.5), # Range -1~1       0~1   0~1
    (-1, 0,   0), (0, 0,   0), (1, 0,   0)
]
state_shape = (96, 96, 1)

reward_history = []

env = gym.make("CarRacing-v2")
agent = DQN(action_space, state_shape)
agent.model.load_weights("models/trial_200o/dqn")
agent.epsilon = agent.epsilon_min

num_episodes = 500
for episode in range(200, num_episodes):
    state, info = env.reset()
    state = process_state_image(state)
    total_reward = 0
    negative_reward = 0
    frame_counter = 0
    done = False

    while not done:
        action = agent.act(state)

        next_state, reward, done, truncated, info = env.step(action)

        next_state = process_state_image(next_state)

        total_reward += reward
        frame_counter += 1

        if frame_counter > 100 and reward < 0:
            negative_reward += 1
        else:
            negative_reward = 0

        if negative_reward > 10 or total_reward < 0:
            done = True # End early

        agent.remember(state, action, reward, next_state, done)
        if frame_counter % 5 == 0:
            agent.replay()

        state = next_state
    
    if episode > 0 and episode % 5 == 0:
        # Save model every 5th episode
        agent.model.save_weights(f"models/trial_{episode}/dqn")
    
    print(f"Episode: {episode+1}, Total Reward: {total_reward}")
    reward_history += [total_reward]

env.close()

Episode: 201, Total Reward: 6.505633802816941
Episode: 202, Total Reward: -0.09072164948453376
Episode: 203, Total Reward: 2.2333333333333654
Episode: 204, Total Reward: 3.9091254752851996
Episode: 205, Total Reward: 1.6795527156549763
Episode: 206, Total Reward: 11.456390977443649
Episode: 207, Total Reward: -0.029889298892965516
Episode: 208, Total Reward: 9.238983050847496
Episode: 209, Total Reward: 5.94545454545458
Episode: 210, Total Reward: 9.91818181818186
Episode: 211, Total Reward: 5.133766233766272
Episode: 212, Total Reward: 0.8047619047619305
Episode: 213, Total Reward: 5.511295681063162
Episode: 214, Total Reward: 3.5250000000000266
Episode: 215, Total Reward: 18.097080291970766
Episode: 216, Total Reward: -0.01098901098898894
Episode: 217, Total Reward: -0.01098901098898894
Episode: 218, Total Reward: 4.9771704180064695
Episode: 219, Total Reward: 9.805923344947773
Episode: 220, Total Reward: 6.50563380281694
Episode: 221, Total Reward: 8.069329073482468
Episode: 222, To

Nach 387 Episoden kam es erneut zu einem Out-Of-Memory-Fehler, sodass ab Episode 385 das Training wieder aufgenommen werden konnte. Zugleich zeichnet sich ab circa Episode 210 ein Anstieg im Reward ab.

In [2]:
def process_state_image(state):
    state = cv2.cvtColor(state, cv2.COLOR_BGR2GRAY)
    state = state.astype(float)
    return state / 255.0

action_space = [
    (-1, 1, 0.5), (0, 1, 0.5), (1, 1, 0.5), # Action Space Structure
    (-1, 1,   0), (0, 1,   0), (1, 1,   0), # (Steering Wheel, Gas, Break)
    (-1, 0, 0.5), (0, 0, 0.5), (1, 0, 0.5), # Range -1~1       0~1   0~1
    (-1, 0,   0), (0, 0,   0), (1, 0,   0)
]
state_shape = (96, 96, 1)

reward_history = []

env = gym.make("CarRacing-v2")
agent = DQN(action_space, state_shape)
agent.model.load_weights("models/trial_385o/dqn")
agent.epsilon = agent.epsilon_min

num_episodes = 500
for episode in range(385, num_episodes):
    state, info = env.reset()
    state = process_state_image(state)
    total_reward = 0
    negative_reward = 0
    frame_counter = 0
    done = False

    while not done:
        action = agent.act(state)

        next_state, reward, done, truncated, info = env.step(action)

        next_state = process_state_image(next_state)

        total_reward += reward
        frame_counter += 1

        if frame_counter > 100 and reward < 0:
            negative_reward += 1
        else:
            negative_reward = 0

        if negative_reward > 10 or total_reward < 0:
            done = True # End early

        agent.remember(state, action, reward, next_state, done)
        if frame_counter % 5 == 0:
            agent.replay()

        state = next_state
    
    if episode > 0 and episode % 10 == 0:
        # Save model every 10th episode
        agent.model.save_weights(f"models/trial_{episode}/dqn")
    
    print(f"Episode: {episode+1}, Total Reward: {total_reward}")
    reward_history += [total_reward]

env.close()

Episode: 386, Total Reward: 74.21329305135961
Episode: 387, Total Reward: 148.42252559727
Episode: 388, Total Reward: 53.41612903225799
Episode: 389, Total Reward: 78.52258064516137
Episode: 390, Total Reward: 85.62222222222238
Episode: 391, Total Reward: 56.5886685552407
Episode: 392, Total Reward: 23.98771929824555
Episode: 393, Total Reward: 17.838906752411496
Episode: 394, Total Reward: 18.42029520295194
Episode: 395, Total Reward: 67.47142857142858
Episode: 396, Total Reward: 8.767549668874212
Episode: 397, Total Reward: 86.837748344371
Episode: 398, Total Reward: 14.459105431309851
Episode: 399, Total Reward: 5.566666666666707
Episode: 400, Total Reward: 56.2758865248226
Episode: 401, Total Reward: 80.15475285171111
Episode: 402, Total Reward: 23.389297658862848
Episode: 403, Total Reward: 65.82307692307695
Episode: 404, Total Reward: 89.8411764705884
Episode: 405, Total Reward: 10.718181818181845
Episode: 406, Total Reward: 18.57359050445095
Episode: 407, Total Reward: 9.1020202

In [3]:
agent.model.save_weights(f"models/trial_500/dqn")

Der Agent erreicht nach 500 Episoden einen durchschnittlichen Reward von 38.